##### Imports

In [2]:
# operations with data
import pandas as pd
# models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# ensembles
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
# visualization
import seaborn as sns
import matplotlib as plt
# preprocessing
import numpy

##### Loading dataset

In [6]:
dataset = pd.read_csv('smoking.csv')
dataset.shape

(55692, 27)

##### Explotary Analysis

#### Creating new hypotheses

#### Preprocessing

##### Models

#### Verifying hypothesis

#### Evaluting results

#### Collecting Results & Submission